In [377]:
import pandas as pd
df = pd.read_csv('tiktok_dataset.csv')
df.head()

,#,claim_status,video_id,video_duration_sec,video_transcription_text,verified_status,author_ban_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,1,claim,7017666017,59,someone shared with me that drone deliveries a...,not verified,under review,343296.0,19425.0,241.0,1.0,0.0
1,2,claim,4014381136,32,someone shared with me that there are more mic...,not verified,active,140877.0,77355.0,19034.0,1161.0,684.0
2,3,claim,9859838091,31,someone shared with me that american industria...,not verified,active,902185.0,97690.0,2858.0,833.0,329.0
3,4,claim,1866847991,25,someone shared with me that the metro of st. p...,not verified,active,437506.0,239954.0,34812.0,1234.0,584.0
4,5,claim,7105231098,19,someone shared with me that the number of busi...,not verified,active,56167.0,34987.0,4110.0,547.0,152.0


In [378]:
df.isna().sum()

#                             0
claim_status                298
video_id                      0
video_duration_sec            0
video_transcription_text    298
verified_status               0
author_ban_status             0
video_view_count            298
video_like_count            298
video_share_count           298
video_download_count        298
video_comment_count         298
dtype: int64

In [379]:
df.dropna(axis=0,inplace=True)
df.shape

(19084, 12)

In [380]:
df.drop(columns=['claim_status','#','video_id','video_transcription_text','author_ban_status'],inplace=True)
df.head()

,video_duration_sec,verified_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,59,not verified,343296.0,19425.0,241.0,1.0,0.0
1,32,not verified,140877.0,77355.0,19034.0,1161.0,684.0
2,31,not verified,902185.0,97690.0,2858.0,833.0,329.0
3,25,not verified,437506.0,239954.0,34812.0,1234.0,584.0
4,19,not verified,56167.0,34987.0,4110.0,547.0,152.0


In [381]:
y = df['verified_status']
df_copy = df.copy()
df_copy.drop(columns='verified_status',axis=1,inplace=True)

In [382]:
x = pd.DataFrame(df_copy,columns=['video_duration_sec','video_view_count','video_like_count','video_share_count','video_download_count','video_comment_count'])
x.head()

,video_duration_sec,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,59,343296.0,19425.0,241.0,1.0,0.0
1,32,140877.0,77355.0,19034.0,1161.0,684.0
2,31,902185.0,97690.0,2858.0,833.0,329.0
3,25,437506.0,239954.0,34812.0,1234.0,584.0
4,19,56167.0,34987.0,4110.0,547.0,152.0


In [383]:
from sklearn.preprocessing import LabelEncoder

labels = LabelEncoder()
y=labels.fit_transform(y)
y = y.reshape(-1,1)[:,0]
y.shape

(19084,)

In [384]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(
    x,
    y,
    stratify=y
)

In [385]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model.fit(x_train,y_train)
log_prediction = log_model.predict(x_test)

In [386]:
log_prediction

array([0, 0, 0, ..., 0, 0, 0])

In [387]:
from sklearn.metrics import accuracy_score, classification_report

print(accuracy_score(y_test,log_prediction))

0.937120100607839


In [388]:
print(classification_report(y_test,log_prediction,target_names=['Non active','active']))

              precision    recall  f1-score   support

  Non active       0.94      1.00      0.97      4471
      active       0.00      0.00      0.00       300

    accuracy                           0.94      4771
   macro avg       0.47      0.50      0.48      4771
weighted avg       0.88      0.94      0.91      4771



c:\Users\maha_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\maha_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\maha_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [389]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
x_smote,y_smote = smote.fit_resample(x,y)

x_smote_train,x_smote_test,y_smote_train,y_smote_test = train_test_split(
    x_smote,
    y_smote
)

In [390]:
smote_model = LogisticRegression()
smote_model.fit(x_smote_train,y_smote_train)
smote_prediction = smote_model.predict(x_smote_test)

In [391]:
print(classification_report(y_smote_test,smote_prediction,target_names=['Non active','active']))

              precision    recall  f1-score   support

  Non active       0.74      0.46      0.56      4462
      active       0.61      0.84      0.71      4480

    accuracy                           0.65      8942
   macro avg       0.67      0.65      0.64      8942
weighted avg       0.67      0.65      0.64      8942



In [392]:
from sklearn.ensemble import RandomForestClassifier

random_model = RandomForestClassifier(n_estimators=200,criterion='log_loss',random_state=42)
random_model.fit(x_train,y_train)
random_model_prediction = random_model.predict(x_test)

In [393]:
print(classification_report(y_test,random_model_prediction,target_names=['Non active','active']))

              precision    recall  f1-score   support

  Non active       0.94      1.00      0.97      4471
      active       0.00      0.00      0.00       300

    accuracy                           0.94      4771
   macro avg       0.47      0.50      0.48      4771
weighted avg       0.88      0.94      0.91      4771



c:\Users\maha_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\maha_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\maha_\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [394]:
random_model_smote = RandomForestClassifier(n_estimators=200,criterion='gini',random_state=42)
random_model_smote.fit(x_smote_train,y_smote_train)
random_model_smote_prediction = random_model_smote.predict(x_smote_test)

In [395]:
print(classification_report(y_smote_test,random_model_smote_prediction,target_names=['Non active','active']))

              precision    recall  f1-score   support

  Non active       0.93      0.95      0.94      4462
      active       0.95      0.93      0.94      4480

    accuracy                           0.94      8942
   macro avg       0.94      0.94      0.94      8942
weighted avg       0.94      0.94      0.94      8942

